<a href="https://colab.research.google.com/github/mmorales20/CS470/blob/master/12_1_DataConverter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

>[Packages](#scrollTo=tkKZZHqhNzuN)

>[Reading in CSV Files](#scrollTo=q7zSBFfcPXG_)

>[Categorical Data Finder](#scrollTo=LX7CdYrqTwe8)

>[Numeric Data Finder](#scrollTo=5yOTozz4UfvX)

>[Random ID Generator](#scrollTo=XCNPi5PvWh2F)

>[SDT Changes](#scrollTo=BeheiUGPVn75)

>[Random Color Generator](#scrollTo=wSVYcrMuWl-U)

>[Runner](#scrollTo=zc5tcEDYWyw5)

>[How to Make a .exe](#scrollTo=Tu14M_iWRnyb)



# Packages


In [0]:
#Download packages
import pandas as pd
import re
import xml.etree.ElementTree as ET

#Download package for File Explorer pop-up
from tkinter import filedialog
from tkinter import *

#Download package for user input pop-up
from tkinter import simpledialog

#Download package to parse through and modify XML tree
from random import randint
from lxml import etree

import os

# Reading in CSV Files

In [0]:
#Display a file explorer and allow user to select a CSV file that contains the data
def selectCSVFile(sdtTemplate):
    root = Tk()
    root.filename =  filedialog.askopenfilename(initialdir = "/",title = "Select CSV file",filetypes = (("text files","*.csv"),("all files","*.*")))
    
    changePath(root.filename,sdtTemplate)
    root.destroy()
    return(root.filename)

 #Test: do not allow user to select any file type other than CSV

In [0]:
#Change the path in the SDT file based on the CSV file selected by the user
def changePath(path,sdtTemplate):
    tree = ET.parse(sdtTemplate)
    root = tree.getroot() 
    
    for elem in root.findall('.//Datasources/Datasource/Host'):
        elem.text=path
    
    for elem in root.findall('.//Datasources/Datasource/Name'):
        elem.text=path
        
    tree.write(sdtTemplate,encoding="utf-8", xml_declaration=True)

In [0]:
#Read in each column of the CSV file that was selected
#INCLUDE VARIABLE IN PROGRAMMERS GUIDE
def readCSV(csvName):
    #df variable is the Pandas dataframe that the csv file is saved to
    df = pd.read_csv(csvName)
    print("reading csv done")
    return df

In [0]:
#Pop-up dialog box to request for user input on how many columns they want to include in their visualization
def userInput():
    root = Tk()
    root.withdraw()

    user_inp = simpledialog.askstring(title="SynGlyphX", 
        prompt="How many columns would you like to include in your visualization?")
    user_int = int(user_inp)
    print("Use", user_int, "columns")
    #add in exception handling if the user input is greater than the number of columns in the CSV file
    return user_int


# Categorical Data Finder

In [0]:
#Add each categorical data column to a dataframe
def getCategoricalColumnName(df,categoricalColumnName):
    x=0
    for name in df.columns:
        if df.dtypes[x] == 'object':
          #FIND OUT WHERE .CAT is FROM
          #CHANGE 101 TO A VARIABLE
             if len(df[name].astype('category').cat.categories.tolist()) >1 and len(df[name].astype('category').cat.categories.tolist()) < 101 and categoricalColumnName=="": 
                categoricalColumnName=df.columns[x]
        x= x+1
    return categoricalColumnName

In [0]:
#Within each column save each unique attribute to a list
def getCategoricalColumnOptions(df,categoricalColumnName):
    return df[categoricalColumnName].astype('category').cat.categories.tolist()

# Numeric Data Finder

In [0]:
#Find numeric (int and float) data and add the column names to a list
def typeDefiner(df,catFound,sdtFile):
    colList = df.columns
    colList
    x = 0
    colNames = []
    for name in colList:
        if df.dtypes[x] == 'object':# or df.dtypes[x] == 'int64':
            pass
        else:
            colNames.append(colList[x])    
        x = x + 1

    #PUT THIS INTO A SEPERATE FUNCTION
    #Finds number of numeric inputs in case of no categorical data
    file = ET.parse(sdtFile)
    root = file.getroot()
    inputCount=0
    for elem in root.findall('.//InputFields'):
        for inputs in elem.findall('.//InputField'):
            inputCount+=1
    
    #PUT THIS INTO A SEPERATE FUNCTION
    #Resets part of color section and input if there is no option for color
    if not catFound and inputCount>len(colNames):
        resetToDefault(sdtFile)
    print("input count is "+str(catFound) +str(inputCount))
    print("objects removed")
    print(colNames)
    return colNames

# Random ID Generator

In [0]:
#Generates random name IDs for one of our subelements
def randomNameIDGenerator():
    name = "~"
    z = 0
    while z < 10:
        value = randint(0, 10)
        name += str(value)
        z = z + 1
    return name  

    #ADD IN EXCEPTION HANDLING

# SDT Changes

In [0]:
#Creates a new SDT file with the same name as the selected CSV file
#Populates the new SDT file with the generic SDT template format
def moveToNewFile(sdtTemplate, csvFile):
    file = ET.parse(sdtTemplate)
    base=os.path.basename(csvFile)
    newFile =os.path.splitext(base)[0]+".sdt"
    file.write(newFile,encoding="utf-8", xml_declaration=True)
    
    return newFile

In [0]:
#ADD IN RUNDOWN OF FUNCTION
def inputFieldGenerator(sdtTemplate):
    tree = ET.parse(sdtTemplate)
    root = tree.getroot()
    fields = tree.find("InputFields")
    colList = typeDefiner(df)
    numberOfCol = userInput()
    
    dataID = tree.find("Datasources")
    datasourceID = dataID[0].attrib['id']

    x = 0
    while x < numberOfCol:
        nameID = randomNameIDGenerator()
        #print('field' + colList[x])
        attrib = {'field': colList[x], 'id' : datasourceID, 'name':nameID, 'table':'OnlyTable', 'type':'Real'}
        element = fields.makeelement('InputField', attrib)
        #ADD IN VARIABLE FOR 6 or EXPLAIN WHY ITS 6
        ET.SubElement(root[6], 'InputField', attrib)
        x = x + 1
    #INCORPORTATE ASHLEYS STUFF
    #BOTTOM SDT FILE IS NEW SDT FILE
    tree.write('testSDT.sdt')

In [0]:
#Clears out the tag in the color section of the SDT file and formats the SDT file
#This method is necessary to use numeric data in the color section
#Without this method all colors would display as the default
def toNumericFormat(updatedSDTFile):

    #Finds keyValuePairs tag and clears out the tag.
    file = ET.parse(updatedSDTFile)
    root = file.getroot()
    for elem in root.findall('.//Glyph/Color/RGB/Function[@type="Text Field To Value"]/KeyValuePairs'):
        elem.clear()

    #Finds end tag and removes end tag from code
    for parent in root.findall('.//KeyValuePairs/..'):
        for element in parent.findall('KeyValuePairs'):
            parent.remove(element)
   #Sets up function to use numeric data with the appropriate lines
    for elem in root.findall('.//Color/RGB/..):
        function_elem = elem.find('.//Function')
        function_elem.set("type","Linear Interpolation")
        attrib = {'type': 'BoundField'}
        minmax = ET.SubElement(check_req_elems, 'MinMax', attrib)
        minmax.text= "\n\t\t\t\t\t\t\t"
        min = ET.SubElement(minmax, 'Min')
        min.text = '0'
        min.tail="\n\t\t\t\t\t\t"
        minmax.tail="\n\t\t\t\t\t"
    file.write(sdtFile)
    print("color fixed")

In [0]:
#Accepted Params: list of column names, the .SDT file, column name, and boolean
# ADD IN OVERVIEW STATEMENT
# RENAME VARIABLES NAMES SO THEY ARE EASIER TO UNDERSTAND AND CONSISTENT
def XMLParser(colNames,sdtFile,colorName,catFound):
    print(colNames)

    tree = ET.parse(sdtFile)
    root = tree.getroot()
    count=0;
    print(len(colNames))
    for elem in root.iter('InputField'):
        print(count)
        if elem.get('name')!="color":
            elem.set('field', colNames[count])
            count+=1
        elif not catFound:
            elem.set('field', colNames[count])
            count+=1
        else:
            elem.set('field', colorName)
        
    tree.write(sdtFile)
    print("XML updated")

In [0]:
#
# ADD IN OVERVIEW STATEMENT
# RENAME VARIABLES NAMES SO THEY ARE EASIER TO UNDERSTAND AND CONSISTENT
# UPDATE TABBING
def setupColor(colorCol,sdtFile):
    file = ET.parse(sdtFile)
    root = file.getroot()
    x=0
    if(colorCol==None):#Changes Color code to take in numeric data
        toNumericFormat(sdtFile)
        
    else:
      #sets up prexisting key value pairs with the first 2 categorical options
        for parent in root.findall('.//Color/RGB/Function[@type="Text Field To Value"]/KeyValuePairs'):
            for elem in parent.findall('KeyValuePair/Key'):
                elem.text=colorCol[x]
                x=x+1

        #Creates new key value pairs for the rest of the categorical list
            if(len(colorCol)>2):
                while x<len(colorCol):
                    for elem in root.findall('.//Color/RGB/Function[@type="Text Field To Value"]'): 
                        pairs = elem.find('.//KeyValuePairs')
                        #fixes spacing so new pair will be at right 
                          #column/indent area
                        #Adds key value pair as subelement to key value pairs
                        for child in pairs.findall('.//KeyValuePair'):
                            child.tail="\n\t\t\t\t\t\t\t"
                        keyVal =ET.SubElement(pairs, 'KeyValuePair')
                        keyVal.text="\n\t\t\t\t\t\t\t\t"
                        keyVal.tail="\n\t\t\t\t\t\t"
                        keyValP= ET.SubElement(keyVal, 'Key')
                        keyValP.text=colorCol[x]
                        keyValP.tail="\n\t\t\t\t\t\t\t\t"
                        key = ET.SubElement(keyVal, 'Value')
                    
                        key.tail="\n\t\t\t\t\t\t\t"
                        x=x+1
    
                        x=0

                        #Creates an array of random colors for the categorical data
                        count=0
                        colorList=[] 
                        while count<len(colorCol):
                            randomCol(colorList)
                            count=count+1

                        #sets up color values for each key value pair
                        for elem in root.findall('.//Color/RGB/Function[@type="Text Field To Value"]/KeyValuePairs'):
                            for element in elem.findall('KeyValuePair/Value'):
                                element.text=colorList[x]
                                print(colorList[x])
                                x=x+1
                
          file.write(sdtFile,encoding="utf-8", xml_declaration=True)


In [0]:
#
# ADD IN OVERVIEW STATEMENT
# MAKE SURE SDT FILE VARIABLE IS WHAT IT SHOULD BE (SDTTemplate or updatedSDTFile)
def resetToDefault(sdtFile):
    file = ET.parse(sdtFile)
    root = file.getroot()
    print("resetting")
    removeElement=None
    for parent in root.findall('.//Glyph/Color/RGB'):
        for elem in parent:
            if(removeElement==None):
                removeElement = parent.find('Binding')
        if(removeElement!=None):
            parent.remove(removeElement)
            removeElement=None
    
    for parent in root.findall('.//InputFields'):
        for elem in parent:
            if(removeElement==None):
                removeElement =  parent.find('InputField[@name="color"]')
              
        if (removeElement!=None):
            parent.remove(removeElement)
            reMoveElement=None
    
    count=0
    #fixes indent spacing for new last element
    for parent in root.findall('.//InputFields'):
        for elem in parent:
            if(count==len(parent)-1):
                elem.tail="\n\t"
            count+=1
        
    file.write(sdtFile,encoding="utf-8", xml_declaration=True)

# Random Color Generator


In [0]:
#Random Color Generator
#Generates random colors and adds them to the color list
def randomColors(colorList):
    import random
    rVal = random.random()*255
    rVal=int(rVal)
    gVal = random.random()*255
    gVal=int(gVal)
    bVal = random.random()*255
    bVal=int(bVal)
    color=str(rVal)+","+str(gVal)+","+str(bVal)
    colorList.append(color)

#Runner

In [0]:
#Uses all necessary methods and variables to run the Glyph Generator
#Converts a CSV file to a SDT file
def runAll(sdtTemplate):
  colorCol = None #will change this name
  categoricalColumnName = ""
  catFound=False  #will change this name
    csvFile = selectCSVFile(sdtTempalte)
    df = readCSV(csvFile)
    categoricalColumnName=getCategoricalColumnName(df,categoricalColumnName)
    if(categoricalColumnName != ""):
        catFound=True   #will change this name
        colorCol=getCategoricalColumnOptions(df,categoricalColumnName) #will change this name
  
    updatedSDTFile=moveToNewFile(sdtTemplate,csvFile)
    categoricalColumnName = typeDefiner(df,catFound,updatedSDTFile) #will change name later
            #typeDefiner will be split into different functions
    XMLParser(colNames,updatedSDTFile,categoricalColumnName,catFound) #make changes to names
    setupColor(colorCol, updatedSDTFile) #will change name later

In [0]:
#Run the Program
  runAll('emptySDT.sdt')

# How to Make a .exe 

Download Python from the web, and select "Add Python 3.7 to PATH"

File -> Download As -> Python (.py)

Open Windows Command Prompt (either look up Windows Command Prompt in Search bar or Windows button + R, 
then type in "cmd")

In Command Prompt - 

> pip install pyinstaller


> cd yourfilepathhere


> pyinstaller --onefile nameofyourfile.py

Then check in the File explorer that were you had your original .py file, and there should be a folder names "dist"

The executable file should be in there

